<a href="https://colab.research.google.com/github/jxxngho/HG_MLDL/blob/main/03_3_Characteristic_engineering_and_Regulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 다중 회귀


In [ ]:
# 다중 회귀 - 여러 개의 특성을 사용한 선형 회귀
# 만약 특성이 두 개면 타깃값과 함께 3차원 공간을 형성하고 선형 회귀 방정식 '타깃 = A * 특성1 + B* 특성2 + 절편
# 특성 공학 - 기존의 특성을 사용해서 새로운 특성을 뽑아내는 작업

# 데이터 준비

In [ ]:
import pandas as pd
df = pd.read_csv('https://bit.ly/perch_csv_data')
perch_full = df.to_numpy()
perch_full

In [ ]:
import numpy as np

perch_weight = np.array([5.9, 32.0, 40.0, 51.5, 70.0, 100.0, 78.0, 80.0, 85.0, 85.0, 110.0,
       115.0, 125.0, 130.0, 120.0, 120.0, 130.0, 135.0, 110.0, 130.0,
       150.0, 145.0, 150.0, 170.0, 225.0, 145.0, 188.0, 180.0, 197.0,
       218.0, 300.0, 260.0, 265.0, 250.0, 250.0, 300.0, 320.0, 514.0,
       556.0, 840.0, 685.0, 700.0, 700.0, 690.0, 900.0, 650.0, 820.0,
       850.0, 900.0, 1015.0, 820.0, 1100.0, 1000.0, 1100.0, 1000.0,
       1000.0])

In [ ]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(perch_full,perch_weight)

# 사이킷런 변환기

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures()
poly.fit([[2,3]])
print(poly.transform([[2,3]]))
# fit 메서드는 새롭게 만들 특성 조합을 찾고
# transform() 메서더는 실제로 데이터를 변환
# 변환기는 입력 데이터를 변환하는데 타깃 데이터가 필요하지 않음

[[1. 2. 3. 4. 6. 9.]]


In [ ]:
poly = PolynomialFeatures(include_bias =False) # 절편 추가 x
poly.fit([[2,3]])
print(poly.transform([[2,3]]))
# 절편을 위한 항이 제거되고 특성의 제곱과 특성끼리 곱한 항만 추가됨

[[2. 3. 4. 6. 9.]]


In [ ]:
# train_input에 적용
poly = PolynomialFeatures(include_bias=False)
poly.fit(train_input)
train_poly = poly.transform(train_input)
print(train_poly.shape)

# 9개 특성 조합 확인
poly.get_feature_names_out()
# x0은 첫번째 특성
# x0^2는 첫번째 특성의 제곱
# x0 x1은 첫번째 특성과 두 번째 특성의 곱

(42, 9)


array(['x0', 'x1', 'x2', 'x0^2', 'x0 x1', 'x0 x2', 'x1^2', 'x1 x2',
       'x2^2'], dtype=object)

In [ ]:
# 테스트 세트로 변환
test_poly = poly.transform(test_input)

# 훈련 세트를 기준으로 테스트 세트를 변환하는 습관을 들이자!

# 다중 회귀 모델 훈련하기

In [ ]:
# 다중 회귀 모델을 훈련하는 것은 선형 회귀 모델을 훈련하는 것과 같다
# 여러 개의 특성을 사용하여 선형 회귀를 사용하는 것!

from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(train_poly,train_target)

# 훈련 세트 score
print(lr.score(train_poly,train_target))

# 테스트 세트 score
print(lr.score(test_poly, test_target))
# 과소적합 문제 해결!

0.988943177595258
0.9866990511566965


In [ ]:
# PolynomialFeatures 클래스의 degree 매개변수를 사용하여 고차항의 최대 차수 지정 가능

poly = PolynomialFeatures(degree=5, include_bias = False)
poly.fit(train_input)
train_poly = poly.transform(train_input)
test_poly =poly.transform(test_input)
print(train_poly.shape)
# 만들어진 특성 55개!

(42, 55)


In [ ]:
lr.fit(train_poly,train_target)
# score 확인

print(lr.score(train_poly,train_target))
print(lr.score(test_poly,test_target))

# 왜 음수?
# 샘플 개수(42)보다 샘플 특성 개수(55)가 많아서 훈련세트에 매~~~우 과대적합된 경우!

0.9999999999996866
-30.395279490688818


# 규제

In [ ]:
# 머신러닝 모델이 훈련 세트를 너무 과도하게 학습하지 못하도록 훼방하는 것